# Self-Supervision

- Train a deep network using the self-supervised learning framework.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import hydra
from omegaconf import OmegaConf
import wandb
import pprint

# custom package
from run_train import check_device_env
from run_ssl_train import prepare_and_run_ssl_train

---

## Specify the dataset, model, and train setting

In [3]:
project = 'caueeg-ssl'
data_cfg_file = 'caueeg-dementia'
train_cfg_file = 'base_train'
ssl_cfg_file = 'byol'
model_cfg_file = '2D-ResNet-50'
device = 'cuda:3'

---

## Initializing configurations using Hydra

In [4]:
with hydra.initialize(config_path='../config'):
    add_configs = [f"data={data_cfg_file}", 
                   f"++data.seq_length=3000",
                   f"++data.input_norm=datapoint",
                   f"++data.awgn=0.05",
                   f"++data.mgn=0.05",
                   f"++data.channel_dropout=0.1",
                   f"++data.crop_multiple=2",
                   f"train={train_cfg_file}",
                   f"+train.device={device}", 
                   f"+train.project={project}",
                   f"++train.lr_scheduler_type=cosine_decay_with_warmup_half",
                   f"++train.search_lr=False",   #####
                   f"++train.base_lr=0.1",       #####
                   f"model={model_cfg_file}",
                   f"++model.minibatch_3090=32",
                   f"++train.save_model=True",
                   f"ssl={ssl_cfg_file}",
                   f"++model.criterion=multi-bce",
                  ]
    
    cfg = hydra.compose(config_name='default', overrides=add_configs)
    
config = {**OmegaConf.to_container(cfg.data), 
          **OmegaConf.to_container(cfg.train),
          **OmegaConf.to_container(cfg.model),
          **OmegaConf.to_container(cfg.ssl)}

check_device_env(config)
pprint.pprint(config)

{'EKG': 'O',
 '_ssl_target_': 'models.ssl.byol.BYOL',
 '_target_': 'models.resnet_2d.ResNet2D',
 'activation': 'mish',
 'awgn': 0.05,
 'awgn_age': 0.001,
 'base_channels': 64,
 'base_lr': 0.1,
 'block': 'bottleneck',
 'channel_dropout': 0.1,
 'conv_layers': [3, 4, 6, 3],
 'criterion': 'multi-bce',
 'crop_multiple': 2,
 'crop_timing_analysis': False,
 'dataset_name': 'caueeg-dataset',
 'dataset_path': 'local/dataset/caueeg-dataset/',
 'ddp': False,
 'device': device(type='cuda', index=3),
 'draw_result': True,
 'dropout': 0.1,
 'embedding_layer': 'pool',
 'fc_stages': 3,
 'file_format': 'memmap',
 'in_channels': '???',
 'input_norm': 'datapoint',
 'latency': 2000,
 'load_event': False,
 'lr_scheduler_type': 'cosine_decay_with_warmup_half',
 'mgn': 0.05,
 'minibatch': 32,
 'minibatch_3090': 32,
 'mixup': 0.0,
 'mlp_hidden_size': 4096,
 'model': '2D-ResNet-50',
 'num_history': 500,
 'out_dims': '???',
 'photic': 'X',
 'project': 'caueeg-ssl',
 'projection_size': 256,
 'run_mode': 'train',

## Train

In [5]:
%%time
prepare_and_run_ssl_train(rank=None, world_size=None, config=config)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
